In [38]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Crew, Task
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'nthu-project-account.json'

In [37]:
from google.cloud import texttospeech

text_block = '''美國職籃（NBA）2024新人選秀今天（27）登場，除了前三名球員之外，外界都很好奇，頂著詹姆斯（LeBron James）之子光環的布朗尼（Bronny James），究竟會在什麼順位，或是被哪隻球隊挑走。稍早第一輪30順位全數出爐，布朗尼並沒有獲得任何球隊青睞，目前仍未被選中。
根據ESPN報導，父親詹姆斯效力的洛杉磯湖人球隊，稍早在第一輪第17順位，挑中來自田納西大學的射手奈契特（Dalton Knecht），並未挑選布朗尼。
從公開資料顯示，奈契特身高6呎6吋（約等於198.1公分），負責後衛位置，來自田納西的他，過去3年繳出不錯的成績，2023-24球季場均攻下21.7分、4.9個籃板，投籃命中率則維持在45.8%、三分命中率也有39.7%的表現。
'''

client = texttospeech.TextToSpeechClient()

synthesis_input = texttospeech.SynthesisInput(text = text_block)

voice = texttospeech.VoiceSelectionParams(
    language_code = "cmn-TW",
    name = 'cmn-TW-Wavenet-A'
)

audio_config = texttospeech.AudioConfig(
    audio_encoding = texttospeech.AudioEncoding.MP3,
    effects_profile_id = ['small-bluetooth-speaker-class-device'],
    speaking_rate = 1,
    pitch = 1
)

response = client.synthesize_speech(
    input = synthesis_input,
    voice = voice,
    audio_config = audio_config
)
with open("output.mp3","wb") as out:
    out.write(response.audio_content)
    print('Audio content written to file"output.mp3"')



Audio content written to file"output.mp3"


In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [9]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [12]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [13]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [14]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [15]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [16]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [17]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [22]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'TSLA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Month Trading',
    'news_impact_consideration': True
}

In [23]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (TSLA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for TSLA to the Data Analyst to generate insights and alerts about significant market opportunities or threats.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for TSLA using statistical modeling and machine learning",
    "context": "The goal is to identify trends and predict market movements to provide insights and alerts about significant market opportunities or threats for TSLA."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for TSLA to identify trends and predict market movements. I should focus on 

In [24]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Proposed Trading Strategies and Execution Plans for TSLA:

After evaluating the approved trading strategies for TSLA, including the Long-Term Growth Strategy, Momentum Trading Strategy, Sector Rotation Strategy, and Risk-Adjusted Strategy, the following comprehensive risk analysis and mitigation recommendations are provided:

1. Long-Term Growth Strategy:
- Potential Risks: Market volatility, macroeconomic factors, regulatory changes, and competitive pressures.
- Mitigation Strategies: Diversification, regular monitoring, and adjusting strategies based on economic conditions.

2. Momentum Trading Strategy:
- Potential Risks: High volatility, price fluctuations, and liquidity risks.
- Mitigation Strategies: Setting stop-loss orders, monitoring positions closely, and avoiding over-leveraging.

3. Sector Rotation Strategy:
- Potential Risks: Sector-specific risks, correlation between sectors, and economic cycles.
- Mitigation Strategies: Thorough research on sector trends, diversification, and balancing exposure.

4. Risk-Adjusted Strategy:
- Potential Risks: Market risks, interest rate fluctuations, currency risks, and geopolitical events.
- Mitigation Strategies: Risk management tools like hedging, derivatives, and maintaining a balanced risk-return profile.

By understanding and addressing the risks associated with each trading strategy, the team can effectively manage risk exposure and maximize potential returns while aligning with the user's risk tolerance and investment goals.